#### Update content of the CPM Atlas - generate maps for future change<br>
Author          : Team BETA<br>
Return Values   : png files<br>
Source data     : The data is preprocessed and provided by Petter Lind.

In [1]:
import numpy as np
import xarray as xr
from pathlib import Path
from itertools import product
# regridding
import xesmf as xe
# plotting
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as crs
from cartopy import feature as cfeature
from textwrap import wrap
# for clipping
import rioxarray

In [2]:
# path to observation datasets
path_data = "/mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind"

In [3]:
# alias of regions
regions_name_dict = {
  "nwe-3": "NW",
  "swe-3": "SW",
  "see-3": "SE",
  "ceu-3": "C",
  "cee-3": "CE",
  "neu-3": "N",
  "alp-3": "AL",
}

# regions name for hclim only
regions_name_dict_hclim = {
  "alp-12": ["C", "AL", "NW", "SW"],
  "eur-11": "N",
  "cee-12": ["CE", "SE"]
}

# define geometries for clipping by regions
regions = {
  "NW": [
    {
      'type': 'Polygon',
      'coordinates': [[[-8.0, 40.4], [11.0, 40.4], [15.2, 58.6], [-12.5, 58.6], [-8.0, 40.4]]]
    }
  ],
  "SW": [
    {
      'type': 'Polygon',
      'coordinates': [[[-10, 30], [7.4, 33], [5.7, 48.9], [-15, 45.4], [-10, 30]]]
    }
  ],
  "SE": [
    {
      'type': 'Polygon',
      'coordinates': [[[12.5, 34.3], [28.5, 34.3], [29.4, 40.9], [11.5, 40.9], [12.5, 34.3]]]
    }
  ],
  "C": [
    {
      'type': 'Polygon',
      'coordinates': [[[5.0, 44.5], [18.0, 45.5], [18.0, 56.0], [1.0, 53.0], [5.0, 44.5]]]
    }
  ],
  "CE": [
    {
      'type': 'Polygon',
      'coordinates': [[[17.8, 41.5], [31.3, 41.5], [32.8, 51.6], [16.4, 51.6], [17.8, 41.5]]]
    }
  ],
  "N": [
    {
      'type': 'Polygon',
      'coordinates': [[[1, 50.7], [26.7, 49.7], [44.1, 70.6], [-9.4, 72.6], [1, 50.7]]]
    }
  ],
  "AL": [
    {
      'type': 'Polygon',
      'coordinates': [[[1, 40], [17, 40], [17, 50], [1, 50], [1, 40]]]
    }
  ]
}

In [4]:
# uniform naming convention consistent with deliverables
# {identifier: [group, model]}
cpm_model_list = {
    "cclm": ["cmcc", "cclm"],
    "cnrm-arome": ["cnrm", "arome41t1"],
    "ethz-cclm": ["ethz", "cclm-gpu"],
    "gerics": ["gerics", "remo"],
    "hclim-knmi": ["knmi", "hclim38-arome"],
    "hclim": ["smhi", "hclim38-arome"],
    "regcm4": ["ictp", "regcm4"],
    "ipsl-wrf": ["ipsl", "wrf"],
    "mohc-um10.1": ["ukmo", "um"]
}

rcm_model_list = {
    "cclm": ["cmcc", "cclm"],
    "cnrm": ["cnrm", "aladin63"],
    "ethz-cclm": ["ethz", "cclm"],
    "gerics": ["gerics", "remo"],
    "knmi-racmo": ["knmi", "racmo"],
    "hclim": ["smhi", "hclim38-aladin"],
    "regcm4": ["ictp", "regcm4"],
    "ipsl-wrf": ["ipsl", "wrf"],
    "mohc-hadgem3-gc3.1": ["ukmo", "hadgem3-gc3.1-n512"]
}

gcm_model_list = {
    "ec-earth-cclm": ["cmcc", "ec-earth"],
    "cnrm": ["cnrm", "cnrm-cm5"],
    "ethz-mpi-esm-lr": ["ethz", "mpi-esm-lr"],
    "gerics": ["gerics", "mpi-esm-lr"],
    "knmi": ["knmi", "ec-earth"],
    "ec-earth-smhi": ["smhi", "ec-earth"],
    "hadgem2-es": ["ictp", "hadgem2-es"],
    "ipsl-cm5a-mr": ["ipsl", "ipsl-cm5"],
    "ipsl-cm6a-lr": ["ipsl", "ipsl-cm6"],
    "glob-25": ["ukmo", "hadgem2-es"]
}

In [5]:
def diff_obs_model(infile, variable):
    """Compute the difference between observation and model outputs.
    """
    # load the input data
    model_data = xr.open_dataset(infile)
    # get the correct observation data
    if variable == "pr":
        obs_data = xr.open_dataset(Path(path_data, "OBS", "EOBS20_seasonal_cycle_pr_day_mean_native_grid_1980-2010_ANN.nc"))
    elif variable == "tas":
        obs_data = xr.open_dataset(Path(path_data, "OBS", "EOBS20_seasonal_cycle_tas_day_mean_native_grid_1980-2010_ANN.nc"))
    #############################################################
    ### regridding from model native grid to observation grid ###
    #############################################################
    # mask input data and observation data
    # note that the mask is the same for all seasons
    model_data['mask'] = xr.where(~np.isnan(model_data[f"{variable}"].sel(season="DJF")), 1, 0)
    obs_data['mask'] = xr.where(~np.isnan(obs_data[f"{variable}"].sel(season="DJF")), 1, 0)
    # create regridder
    regridder = xe.Regridder(model_data, obs_data, "bilinear")
    # use the regridder to regrid data
    model_data_regrid = regridder(model_data[f"{variable}"])
    ###############################################################
    ### compute difference between model output and observation ###
    ###############################################################
    diff_data = model_data_regrid - obs_data[f"{variable}"]

    return diff_data

In [6]:
def region_clip(input_data, region):
    # change filled values to 0, to avoid errors after clipping
    # input_data[f"{variable}"] = input_data[f"{variable}"].where(input_data[f"{variable}"]<1e+10, 0)
    # set-up the coordinate system known to rio
    input_data.rio.write_crs("EPSG:4326", inplace=True)
    # clip data
    clipped_data = input_data.rio.clip(regions[region], "EPSG:4326", all_touched=True)

    return clipped_data

In [7]:
# function for plotting maps
def plot_map(dataset, outfile, project, group, season, region, variable):
    # Plot
    fig = plt.figure(dpi=120)
    ax = fig.add_subplot(111, projection=crs.PlateCarree())
    if variable == "pr":
        dataset.sel(season=f"{season}").plot(ax=ax, vmin=-5, vmax=5, cmap="BrBG",
                                        cbar_kwargs={'label': 'Precipitation difference (mm/day)',
                                        'extend': 'neither', 'location': 'bottom'})
    elif variable == "tas":
        dataset.sel(season=f"{season}").plot(ax=ax, vmin=-6, vmax=6, cmap="coolwarm",
                                        cbar_kwargs={'label': 'Temperature difference (degC)',
                                        'extend': 'neither', 'location': 'bottom'})
    # colorbar
    # cbar = fig.colorbar(im)
    # cbar.set_label('')
    # Prettify
    ax.add_feature(cfeature.OCEAN, zorder=2)
    ax.coastlines(zorder=3)
    ax.axis('off')
    if variable == "pr":
        ax.set_title("\n".join(wrap(f'{group.upper()} {project}| {season} | {region}', 30)))
    elif variable == "tas":
        ax.set_title("\n".join(wrap(f'{group.upper()} {project}| {season} | {region}', 30)))

    # Save
    plt.tight_layout()
    fig.savefig(outfile)
    plt.close()

In [8]:
# load all relevant datasets and plot maps
seasons = ['DJF', 'JJA']
variables = ['tas', 'pr']
# projects = ["CPM", "RCM", "GCM"]
projects = ["GCM"]

for project, variable in product(projects, variables):
    directory = path_data + f"/{project}/{variable}"
    for ncfile in Path(directory).iterdir():
        # filter the historical runs
        if "rcp85" in str(ncfile):
            if project == "GCM":
                name_model_group = ncfile.stem.split('_seasonal')[0]
                # handle the exceptions of file name
                if len(name_model_group.split('_')) == 3:
                    group, model, _ = name_model_group.split('_')
                else:
                    group, _ = name_model_group.split('_')
                # calculate the difference and clip
                diff_data = diff_obs_model(ncfile, variable)
                for region_alias in regions.keys():
                    if group == "ipsl-cm6a-lr" and region_alias != 'SW':
                        pass
                    elif group == "ipsl-cm5a-mr" and region_alias == 'SW':
                        pass
                    else:
                        clipped_region = region_clip(diff_data, region_alias)
                        for season in seasons:
                            outfile = (f"../static/cpm_analysis/future_change/{region_alias}/{variable}/{project.lower()}_{gcm_model_list[group][0]}_{season}.png")
                            plot_map(clipped_region, outfile, project, gcm_model_list[group][0],
                                season, region_alias, variable)
                            print(f"Processed {ncfile}, created {outfile}.")
            if project == "RCM":
                name_region_model_group = ncfile.stem.split('_seasonal')[0]
                if len(name_region_model_group.split('_')) == 4:
                    region, group, model = name_region_model_group.split('_')[:3]
                else:
                    region, group = name_region_model_group.split('_')[:2]
                # calculate the difference and clip
                diff_data = diff_obs_model(ncfile, variable)
                # handle the exception for hclim-ec-earth runs
                if group == "hclim":
                    for region_alias in regions_name_dict_hclim[f"{region}"]:
                        clipped_region = region_clip(diff_data, region_alias)
                        for season in seasons:
                            outfile = (f"../static/cpm_analysis/future_change/{region_alias}/{variable}/{project.lower()}_{rcm_model_list[group][0]}_{season}.png")
                            plot_map(clipped_region, outfile, project, rcm_model_list[group][0],
                                 season, region_alias, variable)
                            print(f"Processed {ncfile}, created {outfile}.")
                # for other data sets try the clipping for all the regions
                else:
                    for region_alias in regions.keys():
                        try:
                            clipped_region = region_clip(diff_data, region_alias)
                            # skip regions with only a few valid points
                            if (len(np.unique(clipped_region.data)) < 15):
                                pass
                            else:
                                for season in seasons:
                                    outfile = (f"../static/cpm_analysis/future_change/{region_alias}/{variable}/{project.lower()}_{rcm_model_list[group][0]}_{season}.png")
                                    plot_map(clipped_region, outfile, project, rcm_model_list[group][0],
                                         season, region_alias, variable)
                                    print(f"Processed {ncfile}, created {outfile}.")
                        except Exception as e:
                            print(e)
                            pass
            if project == "CPM":
                region, group, model = ncfile.stem.split('_')[:3]
                if region == "reu-3": # reu-3 covers all the sub-regions
                    diff_data = diff_obs_model(ncfile, variable)
                    for region_alias in regions.keys():
                        clipped_region = region_clip(diff_data, region_alias)
                        for season in seasons:
                            outfile = (f"../static/cpm_analysis/future_change/{region_alias}/{variable}/{project.lower()}_{cpm_model_list[group][0]}_{season}.png")
                            plot_map(clipped_region, outfile, project, cpm_model_list[group][0], season, region_alias, variable)
                            print(f"Processed {ncfile}, created {outfile}.")
                else: # other regions correspond to single sub-region
                    region_alias = regions_name_dict[f"{region}"]
                    diff_data = diff_obs_model(ncfile, variable)
                    clipped_region = region_clip(diff_data, region_alias)
                    for season in seasons:
                        outfile = (f"../static/cpm_analysis/future_change/{region_alias}/{variable}/{project.lower()}_{cpm_model_list[group][0]}_{season}.png")
                        plot_map(clipped_region, outfile, project, cpm_model_list[group][0],
                         season, region_alias, variable)
                        print(f"Processed {ncfile}, created {outfile}.")

Processed /mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/GCM/tas/cnrm_cnrm-cm5_rcp85_seasonal_cycle_tas_day_mean_native_grid_2041-2050_ANN.nc, created ../static/cpm_analysis/future_change/NW/tas/gcm_cnrm_DJF.png.
Processed /mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/GCM/tas/cnrm_cnrm-cm5_rcp85_seasonal_cycle_tas_day_mean_native_grid_2041-2050_ANN.nc, created ../static/cpm_analysis/future_change/NW/tas/gcm_cnrm_JJA.png.
Processed /mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/GCM/tas/cnrm_cnrm-cm5_rcp85_seasonal_cycle_tas_day_mean_native_grid_2041-2050_ANN.nc, created ../static/cpm_analysis/future_change/SW/tas/gcm_cnrm_DJF.png.
Processed /mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/GCM/tas/cnrm_cnrm-cm5_rcp85_seasonal_cycle_tas_day_mean_native_grid_2041-2050_ANN.nc, created ../static/cpm_analysis/future_change/SW/tas/gcm_cnrm_JJA.png.
Processed /mnt/d/NLeSC/BETA/EUCP/Data_Catalogue/data_Petter_Lind/GCM/tas/cnrm_cnrm-cm5_rcp85_seasonal_cycle_tas_day_